In [ ]:
from pathlib import Path
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

PROJECT_ROOT = Path.cwd().resolve().parents[2]

DATASETS_DIR = PROJECT_ROOT / "datasets"
RESULTS_DIR = PROJECT_ROOT / "results"

TABLES_DIR = RESULTS_DIR / "tables" / "backbone_benchmarking"

MODELS_ROOT = PROJECT_ROOT / "models" / "backbone_benchmark_models"
DEEP_DIR = MODELS_ROOT / "DeepConvNet"
EEGNET_DIR = MODELS_ROOT / "EEGNet"
SHALLOW_DIR = MODELS_ROOT / "ShallowConvNet"

for d in [DEEP_DIR, EEGNET_DIR, SHALLOW_DIR, TABLES_DIR]:
    d.mkdir(parents=True, exist_ok=True)

DATA_PATH = DATASETS_DIR / "physionet_dataset" / "processed" / "preprocessed.npz"

print("Project Root:", PROJECT_ROOT)
print("Tables ->", TABLES_DIR)
print("Models ->", MODELS_ROOT)

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(42)

In [17]:
if not DATA_PATH.exists():
    raise FileNotFoundError(DATA_PATH)

d = np.load(DATA_PATH, allow_pickle=True)
X = d["X"]
y = d["y"].astype(int)

print("X.shape:", X.shape)
print("Label distribution:", dict(zip(*np.unique(y, return_counts=True))))

device = "cuda" if torch.cuda.is_available() else "cpu"

X.shape: (30, 64, 561)
Label distribution: {np.int64(0): np.int64(14), np.int64(1): np.int64(16)}


In [18]:
class EEGDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.int64)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]), torch.tensor(self.y[idx])

In [19]:
class EEGNet(nn.Module):
    def __init__(self, chans, samples, classes=2):
        super().__init__()
        self.first = nn.Sequential(
            nn.Conv2d(1, 8, (1, 64), padding=(0, 32), bias=False),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 16, (chans, 1), bias=False),
            nn.BatchNorm2d(16),
            nn.ELU(),
            nn.AvgPool2d((1,4)),
            nn.Dropout(0.5)
        )
        self.second = nn.Sequential(
            nn.Conv2d(16, 16, (1, 16), bias=False),
            nn.BatchNorm2d(16),
            nn.ELU(),
            nn.AvgPool2d((1,8)),
            nn.Flatten()
        )

        with torch.no_grad():
            dummy = torch.zeros(1,1,chans,samples)
            feat = self.second(self.first(dummy))
            dim = feat.shape[1]

        self.classifier = nn.Linear(dim, classes)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.first(x)
        x = self.second(x)
        return self.classifier(x)

In [20]:
class DeepConvNet(nn.Module):
    def __init__(self, chans, samples, classes=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 25, (1, 5)),
            nn.Conv2d(25, 25, (chans, 1)),
            nn.BatchNorm2d(25),
            nn.ELU(),
            nn.MaxPool2d((1, 2)),
            nn.Dropout(0.5),

            nn.Conv2d(25, 50, (1, 5)),
            nn.BatchNorm2d(50),
            nn.ELU(),
            nn.MaxPool2d((1, 2)),
            nn.Dropout(0.5),

            nn.Flatten()
        )

        with torch.no_grad():
            dummy = torch.zeros(1,1,chans,samples)
            feat = self.net(dummy)
            dim = feat.shape[1]

        self.classifier = nn.Linear(dim, classes)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.net(x)
        return self.classifier(x)

In [21]:
class ShallowConvNet(nn.Module):
    def __init__(self, chans, samples, classes=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 40, (1, 13)),
            nn.Conv2d(40, 40, (chans, 1)),
            nn.BatchNorm2d(40),
            nn.ELU(),
            nn.AvgPool2d((1, 35)),
            nn.Dropout(0.5),
            nn.Flatten()
        )

        with torch.no_grad():
            dummy = torch.zeros(1,1,chans,samples)
            feat = self.net(dummy)
            dim = feat.shape[1]

        self.classifier = nn.Linear(dim, classes)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.net(x)
        return self.classifier(x)

In [ ]:
def train_kfold(model_class, model_name, save_dir, X, y, epochs=25):

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    results = []

    for fold, (tr, te) in enumerate(skf.split(X, y), 1):

        ds_tr = EEGDataset(X[tr], y[tr])
        ds_te = EEGDataset(X[te], y[te])

        loader_tr = DataLoader(ds_tr, batch_size=16, shuffle=True)
        loader_te = DataLoader(ds_te, batch_size=32, shuffle=False)

        model = model_class(X.shape[1], X.shape[2], classes=len(np.unique(y))).to(device)
        opt = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
        loss_fn = nn.CrossEntropyLoss()

        best_acc = 0.0
        best_state = None

        for ep in range(epochs):

            model.train()
            for xb, yb in loader_tr:
                xb = xb.to(device)
                yb = yb.to(device)

                opt.zero_grad()
                logits = model(xb)
                loss = loss_fn(logits, yb)
                loss.backward()
                opt.step()

            model.eval()
            preds, gts = [], []
            with torch.no_grad():
                for xb, yb in loader_te:
                    xb = xb.to(device)
                    logits = model(xb)
                    preds.extend(logits.argmax(dim=1).cpu().numpy())
                    gts.extend(yb.numpy())

            acc = accuracy_score(gts, preds)

            if acc > best_acc:
                best_acc = acc
                best_state = {k: v.clone() for k, v in model.state_dict().items()}

        model.load_state_dict(best_state)

        model_path = save_dir / f"{model_name.lower()}_fold{fold}_best.pth"
        torch.save(model.state_dict(), model_path)

        print(f"{model_name} Fold {fold} Best Acc: {best_acc:.3f}")

        results.append({"fold": fold, "accuracy": best_acc})

    df = pd.DataFrame(results)
    df["mean_accuracy"] = df["accuracy"].mean()

    df.to_csv(TABLES_DIR / f"{model_name.lower()}_kfold_results.csv", index=False)

    return df

In [23]:
deep_results = train_kfold(DeepConvNet, "DeepConvNet", DEEP_DIR, X, y)
eeg_results = train_kfold(EEGNet, "EEGNet", EEGNET_DIR, X, y)
shallow_results = train_kfold(ShallowConvNet, "ShallowConvNet", SHALLOW_DIR, X, y)

DeepConvNet Fold 1 Best Acc: 0.667
DeepConvNet Fold 2 Best Acc: 0.667
DeepConvNet Fold 3 Best Acc: 0.667
DeepConvNet Fold 4 Best Acc: 1.000
DeepConvNet Fold 5 Best Acc: 0.667
EEGNet Fold 1 Best Acc: 0.667
EEGNet Fold 2 Best Acc: 0.833
EEGNet Fold 3 Best Acc: 0.667
EEGNet Fold 4 Best Acc: 0.833
EEGNet Fold 5 Best Acc: 0.833
ShallowConvNet Fold 1 Best Acc: 0.667
ShallowConvNet Fold 2 Best Acc: 0.500
ShallowConvNet Fold 3 Best Acc: 0.667
ShallowConvNet Fold 4 Best Acc: 0.667
ShallowConvNet Fold 5 Best Acc: 0.500
